In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = pd.read_pickle("./ny_data_cleaned.pkl")

In [3]:
df

,date,tmax_vc,tmax_om,tmax_ms,tmax_ncei,humi_vc,prec_om,tmin_ms,tmin_ncei,day,tmax_avg,tmin_avg
0,2020-01-15,50.2,48.727398,50.00,50.00,66.4,3.0,42.80,41.00,15,49.73185,41.90
1,2020-01-16,48.2,47.647400,48.20,48.92,63.7,8.0,33.80,33.08,16,48.24185,33.44
2,2020-01-17,34.1,36.757400,33.80,33.08,36.0,0.0,21.20,21.20,17,34.43435,21.20
3,2020-01-18,37.1,32.797400,37.40,37.04,60.2,6.0,19.40,20.12,18,36.08435,19.76
4,2020-01-19,42.4,44.227400,42.80,42.98,62.2,7.0,28.40,27.14,19,43.10185,27.77
...,...,...,...,...,...,...,...,...,...,...,...,...
1442,2024-03-04,57.7,58.267398,61.70,59.00,71.0,0.0,46.04,46.04,64,59.16685,46.04
1443,2024-03-05,47.9,46.747400,48.02,48.92,87.6,16.0,42.80,42.08,65,47.89685,42.44
1444,2024-03-06,52.6,49.987400,52.70,53.06,86.1,6.0,42.26,46.94,66,52.08685,44.60
1445,2024-03-07,53.9,54.757400,51.62,53.96,73.9,13.0,41.72,44.06,67,53.55935,42.89


In [4]:
# Assuming the relationship between today's weather and tomorrow's max temp, shift tmax
df['tmax_tomorrow'] = df['tmax_avg'].shift(-1)

# Drop the last row since it has no tomorrow's tmax
df = df[:-1]

# Optionally, convert 'date' to numerical day of year if you think it's predictive
df['day'] = df['date'].dt.dayofyear

# Selecting features and target
X = df[['day', 'tmax_avg', 'tmin_avg', 'prec_om', 'humi_vc']]
y = df['tmax_tomorrow']

/var/folders/b9/fkf035lj1y786gv5xh300_qc0000gn/T/ipykernel_61389/4031795437.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day'] = df['date'].dt.dayofyear


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [7]:
# Predicting on the testing set
y_pred = model.predict(X_test)

# Calculating the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 49.235347186679526


In [8]:
today_features = np.array([[200, 56, 54, 3, 52]])  # Example: day_of_year, tmax, tmin, precipitation, humidity
tomorrow_tmax_pred = model.predict(today_features)
print(f"Tomorrow's predicted Tmax: {tomorrow_tmax_pred[0]:.2f}°C")

Tomorrow's predicted Tmax: 62.72°C


/opt/miniconda3/envs/cs542/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
